### Santander Coders 2023 | 2º semestre - DS (Turma #1114)

**Dupla:** Mariana Cassundé e Suzielli Mendonça

**Projeto**  
Banco de Dados I   
Análise Descritiva do Conjunto de Dados do Titanic

**Objetivo:**  
Fazer uma Análise Descritiva de um conjunto de dados para aplicar os conhecimentos adquiridos sobre Structured Query Language ([SQL](https://en.wikipedia.org/wiki/SQL#:~:text=Structured%20Query%20Language%20(SQL)%20(,database%20management%20system%20(RDBMS)))) utilizando um conjunto de dados sobre passageiros do Titanic.

[Link para o conjunto de dados](https://www.kaggle.com/datasets/brendan45774/test-file)    


**Sobre o conjunto de dados:**    
O Titanic foi um navio britânico de passageiros que afundou após colidir com um iceberg durante sua viagem inaugural de Southampton para Nova York. Muitos apssageiros morreram por conta desta colisão, tornando-o um dos desastres marítimos mais mortais em tempos de paz na história.

Conjuntos de dados sobre o Titanic são frequentemente usados em competições de ciência de dados e machine learning, especialmente para iniciantes, como um exercício para praticar habilidades de limpeza de dados, análise exploratória e modelagem preditiva.  

O conjunto de dados utilizado para [essa análise](https://www.kaggle.com/datasets/brendan45774/test-file) é uma combinação do arquivo de teste originalmente usado na competição do Titanic no Kaggle e o arquivo de submissão de gênero, que juntos fornecem informações sobre os passageiros, incluindo se eles sobreviveram ou não ao desastre. O arquivo .csv foi criado por um usuário da plataforma [Kaggle](https://www.kaggle.com/). Os dados fornecidos no Data Set em tela não necessariamente reflete a veracidade dos fatos referentes ao acidente ocorrido em 1912.

____

**Bibliotecas utilizadas no projeto:**    
- Pandas
- PandaSQL
- SQLAlchemy
- Psycopg2 (database adpater for Python)

---

**Algumas perguntas que gostaríamos de responder a partir desta análise**:    
1. Quantos passageiros embarcaram em cada porto?    
2. Qual foi a tarifa média paga por passageiros de cada classe (dólares)?   
3. Qual a idade do passageiro mais jovem e do mais velho a bordo (anos)?   
4. Quantos passageiros são do sexo masculino e quantos do sexo feminino?   
5. Quantos passageiros sobreviveram e quantos não sobreviveram?   
6. Qual a distribuição de passageiros por faixa etária e sobrevivência?   
7. Qual a média de idade dos passageiros que sobreviveram (em anos)?   
8. Qual a classe de passagem tinha o maior número de sobreviventes?   
9. Qual a média de tarifa paga por passageiros que sobreviveram versus os que não sobreviveram?   
10. Quem foi o(a) passageiro(a) mais velho(a) que sobreviveu?  

---

**Importação de bibliotecas e carregamento do conjunto de dados**

In [2]:
import pandasql as ps
import sqlalchemy
import psycopg2
import pandas as pd

dados = pd.read_csv("tested.csv")

ps.sqldf("SELECT * FROM dados LIMIT 10")

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,None,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,None,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,None,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,None,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,None,S
5,897,0,3,"Svensson, Mr. Johan Cervin",male,14.0,0,0,7538,9.2250,None,S
6,898,1,3,"Connolly, Miss. Kate",female,30.0,0,0,330972,7.6292,None,Q
7,899,0,2,"Caldwell, Mr. Albert Francis",male,26.0,1,1,248738,29.0000,None,S
8,900,1,3,"Abrahim, Mrs. Joseph (Sophie Halaut Easu)",female,18.0,0,0,2657,7.2292,None,C
9,901,0,3,"Davies, Mr. John Samuel",male,21.0,2,0,A/4 48871,24.1500,None,S


**Descrição das colunas:**    
- **PassengerId:** ID do passageiro;    
- **Survived:** Se o passageiro sobreviveu (1) ou não (0).
- **Pclass:** Classe do bilhete (1, 2, 3).
- **Name:** Nome do passageiro.
- **Sex:** Sexo do passageiro.
- **Age:** Idade do passageiro.
- **SibSp:** Número de irmãos/cônjuges a bordo.
- **Parch:** Número de pais/filhos a bordo.
- **Ticket:** Número do bilhete.
- **Fare:** Tarifa paga pelo passageiro.
- **Cabin:** Número da cabine.
- **Embarked:** Porto de embarque (C = Cherbourg; Q = Queenstown; S = Southampton).

**Verificação da tipagem das colunas**

In [4]:
print(dados.dtypes)

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object


**Verificação de valores faltantes**

In [9]:
query = """
SELECT
    SUM(CASE WHEN PassengerId IS NULL THEN 1 ELSE 0 END) AS NullsPassengerId,
    SUM(CASE WHEN Survived IS NULL THEN 1 ELSE 0 END) AS NullsSurvived,
    SUM(CASE WHEN Pclass IS NULL THEN 1 ELSE 0 END) AS NullsPclass,
    SUM(CASE WHEN Name IS NULL THEN 1 ELSE 0 END) AS NullsName,
    SUM(CASE WHEN Sex IS NULL THEN 1 ELSE 0 END) AS NullsSex,
    SUM(CASE WHEN Age IS NULL THEN 1 ELSE 0 END) AS NullsAge,
    SUM(CASE WHEN SibSp IS NULL THEN 1 ELSE 0 END) AS NullsSibSp,
    SUM(CASE WHEN Parch IS NULL THEN 1 ELSE 0 END) AS NullsParch,
    SUM(CASE WHEN Ticket IS NULL THEN 1 ELSE 0 END) AS NullsTicket,
    SUM(CASE WHEN Fare IS NULL THEN 1 ELSE 0 END) AS NullsFare,
    SUM(CASE WHEN Cabin IS NULL THEN 1 ELSE 0 END) AS NullsCabin,
    SUM(CASE WHEN Embarked IS NULL THEN 1 ELSE 0 END) AS NullsEmbarked
FROM dados;
"""
resultado = ps.sqldf(query, locals())
resultado


,NullsPassengerId,NullsSurvived,NullsPclass,NullsName,NullsSex,NullsAge,NullsSibSp,NullsParch,NullsTicket,NullsFare,NullsCabin,NullsEmbarked
0,0,0,0,0,0,86,0,0,0,1,327,0


- As colunas **Age**, **Cabin** e **Fare** possuem valores faltantes

**Verificação de duplicatas**

In [10]:
query = """
SELECT 
    CASE 
        WHEN COUNT(PassengerId) > 1 THEN COUNT(PassengerId) - 1
        ELSE 0 
    END AS DuplicatesPassengerId
FROM dados
WHERE PassengerId IN (
    SELECT PassengerId 
    FROM dados 
    GROUP BY PassengerId 
    HAVING COUNT(PassengerId) > 1
);
"""
resultado = ps.sqldf(query, locals())
resultado

,DuplicatesPassengerId
0,0


**1. Quantos passageiros embarcaram em cada porto?**    
C = Cherbourg Q = Queenstown S = Southampton

In [11]:
query = """
SELECT 
    Embarked, 
    COUNT(*) AS Count
FROM dados 
GROUP BY Embarked 
ORDER BY Count DESC
"""

resultado = ps.sqldf(query, locals())
resultado

,Embarked,Count
0,S,270
1,C,102
2,Q,46


**2. Qual foi a tarifa média paga por passageiros de cada classe (dólares)?**

In [12]:
query = """
SELECT 
    Pclass, 
    AVG(Fare) AS AverageFare
FROM dados 
GROUP BY Pclass
"""

resultado = ps.sqldf(query, locals())
resultado

,Pclass,AverageFare
0,1,94.280297
1,2,22.202104
2,3,12.459678


**3. Qual a idade do passageiro mais jovem e do mais velho a bordo (anos)?**

In [13]:
query = """
SELECT 
    MIN(Age) AS Youngest,
    MAX(Age) AS Oldest
FROM dados
"""

resultado = ps.sqldf(query, locals())
resultado

,Youngest,Oldest
0,0.17,76.0


**4. Quantos passageiros são do sexo masculino e quantos do sexo feminino?**

In [14]:
query = """
SELECT 
    Sex, 
    COUNT(*) AS Count
FROM dados 
GROUP BY Sex
"""

resultado = ps.sqldf(query, locals())
resultado

,Sex,Count
0,female,152
1,male,266


**5. Quantos passageiros sobreviveram e quantos não sobreviveram?**    
0 = Não Sobrevivente
1 = Sobrevivente

In [15]:
query = """
SELECT 
    Survived, 
    COUNT(*) AS Count
FROM dados 
GROUP BY Survived
"""

resultado = ps.sqldf(query, locals())
resultado

,Survived,Count
0,0,266
1,1,152


**6. Qual a distribuição de passageiros que sobreviveram por faixa etária?**

In [3]:
query = """
SELECT 
    CASE 
        WHEN Age <= 18 THEN '0-18'
        WHEN Age > 18 AND Age <= 60 THEN '19-60'
        WHEN Age > 60 THEN '60+'
        ELSE 'Unknown'
    END AS AgeGroup,
    Survived,
    COUNT(*) AS Count
FROM dados 
WHERE Survived = 1
GROUP BY AgeGroup, Survived
"""

resultado = ps.sqldf(query, locals())
resultado

,AgeGroup,Survived,Count
0,0-18,1,24
1,19-60,1,99
2,60+,1,4
3,Unknown,1,25


**7. Qual a média de idade dos passageiros que sobreviveram (em anos)?**

In [17]:
query = """
SELECT 
    AVG(Age) AS avg_age_survivors
FROM dados 
WHERE Survived = 1
"""

resultado = ps.sqldf(query, locals())
resultado

,avg_age_survivors
0,30.272362


**8. Qual classe de passagem tinha o maior número de sobreviventes?**    
1 = primeira classe
2 = segunda classe
3 = terceira classe

In [7]:
query = """
SELECT 
    Pclass AS CabinClass, 
    COUNT(*) AS Survivors
FROM dados 
WHERE Survived = 1
GROUP BY Pclass
ORDER BY Survivors DESC
"""

resultado = ps.sqldf(query, locals())
resultado

,CabinClass,Survivors
0,3,72
1,1,50
2,2,30


**9. Qual a média de tarifa paga por passageiros que sobreviveram versus os que não sobreviveram?**

In [20]:
query = """
SELECT 
    Survived, 
    AVG(Fare) AS AverageFare
FROM dados 
GROUP BY Survived
"""

resultado = ps.sqldf(query, locals())
resultado

,Survived,AverageFare
0,0,27.527877
1,1,49.747699


**10. Quem foi o(a) passageiro(a) mais velho(a) que sobreviveu?**

In [22]:
query = """
SELECT
    Name,
    Sex, 
    MAX(Age) AS MaxAge
FROM dados 
WHERE Survived = 1 
"""

resultado = ps.sqldf(query, locals())
resultado

,Name,Sex,MaxAge
0,"Cavendish, Mrs. Tyrell William (Julia Florence...",female,76.0


**Conclusão**

Do conjunto de dados extraídos do arquivo .csv selecionado, cujo tema aborda o acidente ocorrido com o navio Titanic em abril do ano de 1912, conclui-se que a embarcação possuía passageiros das mais variadas faixas etárias, sendo o mais jovem possuindo menos de um ano de idade e o mais velho na casa dos 76 anos, sendo a grande maioria pertencente ao sexo masculino.    
A maior parte dos viajantes partiu do Porto de Southampton, embora alguns tenham embarcado de Cherbourg e também de Queenstown, este em sua minoria. Enquanto a média de idade dos sobreviventes foi de aproximadamente 30 anos, apenas 4 idosos e 24 crianças conseguiram sair com vida.    
Por fim, tendo em vista que a média do valor pago para desfrutar do conforto de viajar de primeira classe foi de aproximadamente 94 dólares, isso não foi o suficiente para garantir um maior índice de sobreviência, já que a maioria dos que resistiram foram passageiros da terceira classe, os quais pagaram em média aproximadamente 12 dólares pela cabine. Titanic é, de longe, o caso de naufrágio mais famoso do mundo, tendo inclusive sido retratado em filme no ano de 1997.

---

**Fim do projeto**